In [148]:
import os
import torch
import torchvision
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import cv2
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from datetime import datetime
import numpy as np
import os
import random
import argparse
from PIL import Image
import csv

In [149]:
data = pd.read_csv(os.path.join("db_lisa_tiny", "annotations.csv"))
data.iloc[0][1]

190

In [150]:
class2id = {"stop": 1, "yield": 2, "yieldAhead": 3, "merge": 4, "signalAhead": 5, "pedestrianCrossing": 6, "keepRight": 7, "speedLimit35": 8, "speedLimit25":9}
# dataset definition
class myDataset(Dataset):
    # load the dataset
    def __init__(self,root,transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "imgs"))))
        #with open(os.path.join(self.root, "annotations.csv")) as file:
        #    self.data = csv.reader(self.data)
        self.data = pd.read_csv(os.path.join(self.root, "annotations.csv"))
    # get a row at an index
    def __getitem__(self, idx):
        idx = idx -1
        img_path = os.path.join(self.root, "imgs", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        
        row = self.data.iloc[idx]
        
        boxes = []
        
        x1 = int(row[1])
        y1 = int(row[2])
        x2 = int(row[3])
        y2 = int(row[4])
        label = class2id[row[5]]#int(row[5])
        boxes.append([x1, y1, x2, y2])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.tensor([label], dtype=torch.int64)
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = torch.tensor([False])
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [151]:
import detection.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.Normalize())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [152]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [153]:
num_classes = 9

In [154]:
dataset = myDataset('db_lisa_tiny', get_transform(train=True))
dataset_test = myDataset('db_lisa_tiny', get_transform(train=False))

In [155]:
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

In [156]:
import detection.utils as utils
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=utils.collate_fn)

In [157]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn()

In [158]:
model = model.to(device)

In [159]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)

In [160]:
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

In [161]:
num_epochs = 8

In [162]:
from detection.engine import train_one_epoch, evaluate

for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

Epoch: [0]  [  0/425]  eta: 0:01:52  lr: 0.000017  loss: 5.1665 (5.1665)  loss_classifier: 4.4604 (4.4604)  loss_box_reg: 0.0058 (0.0058)  loss_objectness: 0.6925 (0.6925)  loss_rpn_box_reg: 0.0077 (0.0077)  time: 0.2648  data: 0.0320  max mem: 1173
Epoch: [0]  [ 10/425]  eta: 0:00:54  lr: 0.000135  loss: 5.1535 (5.1243)  loss_classifier: 4.4479 (4.4153)  loss_box_reg: 0.0068 (0.0113)  loss_objectness: 0.6939 (0.6937)  loss_rpn_box_reg: 0.0036 (0.0041)  time: 0.1317  data: 0.0229  max mem: 1238
Epoch: [0]  [ 20/425]  eta: 0:00:50  lr: 0.000252  loss: 4.9589 (4.8708)  loss_classifier: 4.2498 (4.1623)  loss_box_reg: 0.0072 (0.0104)  loss_objectness: 0.6929 (0.6933)  loss_rpn_box_reg: 0.0038 (0.0048)  time: 0.1170  data: 0.0221  max mem: 1238
Epoch: [0]  [ 30/425]  eta: 0:00:47  lr: 0.000370  loss: 3.4360 (3.9298)  loss_classifier: 2.7280 (3.2239)  loss_box_reg: 0.0075 (0.0097)  loss_objectness: 0.6914 (0.6914)  loss_rpn_box_reg: 0.0048 (0.0048)  time: 0.1152  data: 0.0226  max mem: 1238


In [163]:
torch.save(model,"model_real_fasterrcnn_mobilenet_v3_large_fpn")

In [164]:
trainedModel = torch.load("model_real_fasterrcnn_mobilenet_v3_large_fpn")

In [165]:
trainedModel.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): ConvNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block): Seque

In [166]:
img, targets = next(iter(data_loader_test))

In [167]:
trainedModel(img)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
imgs = img[3]
imgs = np.transpose(imgs, (1,2,0)) #tt = np.transpose(tt,(1,2,0))
plt.figure()
plt.imshow(imgs)

plt.show()